In [1]:
#load "sig.fsx"
open CSCI374.ExtraReflection

The below script needs to be able to find the current output cell; this is an easy method to get it.

# F# Programming: Functions

#  Lambda Calculus
![lambda](img/lambda.png)

#  Lambda Calculus (cont.)

- Equivalent to Turing machines
- Instead of state, we use **pure functions**
- Functional programming languages based on this
- Only three rules:
    - variables
    - function definitions
    - function applications 
- Example: λx. λy. x+y

## Lambda Function

Notation:
- Variable, $e$
- Function, $\lambda x.e$
    - the body of a lambda extends as far as possible to the right
    - $\lambda x.x \lambda z.x z x$ is $\lambda x.(x \lambda z.(x z x))$ 
- Function call, $e\, e$
    - applications associate to the left
    - `y z x` is `(y z) x`

Example:
- Identity function, $\lambda x.x$
- Addition function, $\lambda x. \lambda y.x+y$  

## Anonymous Functions

F# provides an alternative way to define functions using the keyword `fun`; you can see this in the following example.
Ordinarily, you would use this notation when it is not necessary to give a name to a function, so these are referred to as **anonymous functions**
and sometimes called **lambda functions** or even just **lambdas**, see [Lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus).

In [1]:
function n -> n * n * n

In [4]:
fun n -> n * n * n

<fun:it@1-1> : (int -> int)

In [89]:
(fun n -> n * n * n) 3 + 1

28

## Functions

Create typical functions using the `let` syntax:

In [90]:
let f = (fun n -> n * n * n) // bind lambda expression to symbol

f 3

27

In [5]:
let printName aName =
    printfn "Hello %s" aName

In [6]:
// test
printName "Alice"

Hello Alice


In [95]:
// define another function
let add x y = x + y

// test
sgn add

Function: x:int -> y:int -> int

In [94]:
sgn (add 1)

Function: y:int -> int

In [5]:
sgn (add 1 2)

Value: int

## Different ways to define a function

In [5]:
let add1 x =
    x + 1   // no "return" keyword. Last expression is "returned"

add1 41  // Result => 42

42

In [10]:
let add1 x =
    let result = x + 1
    result    // return result

add1 41  // Result => 42

42

We can define function using an anonymous function. An anonymous function (or "lambda expression") is defined using the form:

    fun parameter1 parameter2 etc -> expression

In [98]:
// let add1 = (fun x -> x + 1)
    
let add1 =
    let result = (fun x -> x + 1)    
    result
    
add1 41 // Result => 42

42

*Q.* Rewrite `let add x y = x + y` using lambdas. There are two alternatives. Compare the signatures of these alternative functions with the original.

## Type Annotations

F# compiler may occasionally need  more information, which we provide in form of **type annotations**.

In [11]:
let add x y = x + y
sgn add

Function: x:int -> y:int -> int

In [12]:
let add x (y:float) = x + y // type annotaton of parameter y
sgn add

Function: x:float -> y:float -> float

In [13]:
let add x y :float = x + y // type annotation of return type
sgn add

Function: x:float -> y:float -> float

## Type Inference

In the absence of additional type information, the following function is assumed to work with integers:

In [42]:
let squareAndAdd a b = a * a + b
squareAndAdd

<fun:it@2-19> : (int -> (int -> int))

A single type annotation on a is sufficient to indicate that `a * a` is an operation on `float` values and
thus returns a `float` value, and that `a * a + b` is also an operation on `float`:

In [43]:
let squareAndAdd (a:float) b = a * a + b
squareAndAdd

<fun:it@2-20> : (double -> (double -> double))

In general, you can place such annotations on any of the function arguments or directly when you use
them in the body of the function. If you want, you can also give full type annotations for the arguments
and return type of a function:

In [44]:
let squareAndAdd (a:float) (b:float) : float = a * a + b
squareAndAdd

<fun:it@2-21> : (double -> (double -> double))

## Composition

In [7]:
let double (x:float) = x + x
sgn double

Function: x:float -> float

In [10]:
let square (x:float) = x * x
sgn square

Function: x:float -> float

In [99]:
sgn (double >> square)

Function: x:float -> float

In [100]:
(double >> square) 2.0

16

In [101]:
let double_square = double >> square
double_square 3.0

36

**Building with composition:** Composition is the *glue* that allows us build larger systems from smaller ones. This is not an optional technique, but is at the very heart of the functional style.

In [104]:
sgn (List.map double)

Function: list:list<float> -> list<float>

In [22]:
// functions as values
[1.0..4.0] |> List.map square

index,value
0,1
1,4
2,9
3,16


In [54]:
// functions taking other functions as parameters
let execFunction aFunc aParam = aFunc aParam
let result2 = execFunction square 12.0
result2

144.0

## Example: Decorator

Create an input log function and an output log function and then use them to create "logged" version of `add1`

In [7]:
let add1 x = x + 1
sgn add1

Function: x:int -> int

In [3]:
let logIn f x =
    printf "In=%A; " x
    f x
sgn logIn

Function: f:(obj -> obj) -> x:obj -> obj

In [5]:
let logOut f x =
    let res = f x
    printf "Out=%A; " res
    res
sgn logOut

Function: f:(obj -> obj) -> x:obj -> obj

In [6]:
sgn (logIn add1) |> printf "%s\n"
logOut (logIn add1) 10

Function: x:int -> int
In=10; Out=11; 

11

In [9]:
sgn (logIn >> logOut) |> printf "%s\n"
sgn ((logIn >> logOut) add1) |> printf "%s\n"
sgn ((logIn >> logOut) add1 10) |> printf "%s\n"

Function: x:(obj -> obj) -> obj -> obj
Function: x:int -> int
In=10; Out=11; Value: int


In [147]:
(add1 |> logIn |> logOut) 10

In=10; Out=11; 

11

In [13]:
let logfunc (f:'a->'a) = (logIn >> logOut) f

In [17]:
logfunc add1 103

In=103; Out=104; 

104

In [18]:
logfunc (fun n -> n*n*n) 4.9

In=4.9; Out=117.649; 

117.64900000000003

In [20]:
[5..10] |> List.map (logfunc add1)

In=5; Out=6; In=6; Out=7; In=7; Out=8; In=8; Out=9; In=9; Out=10; In=10; Out=11; 

index,value
0,6
1,7
2,8
3,9
4,10
5,11


## Piping

In [2]:
let add2 x = x + 2
sgn add2

Function: x:int -> int

Let's try to call call several functions, passing results of some functions to inputs of others.

In [3]:
(add2 3)

5

In [4]:
float (add2 3)

5

In [8]:
double (float (add2 3))

10

In [15]:
(square (double (float (add2 3)))) + 1.0

101

First function to call is the last function written, `add2`, and the last function to call is written first `square`.
We can fix a written order of function calls with **piping**.

In [16]:
// square (double (float (add2 3))) + 1.0
3 |> add2 |> float |> double |> square |> (+) 1.0

101

## Parameter-less functions

Let's have a function that prints a message

In [17]:
let sayHello = printfn "Hello World!"     // not what we want
//val sayHello : unit = ()

Hello World!


In [18]:
sayHello

The fix is to use a lambda

In [19]:
let sayHello = fun () -> printfn "Hello World!"   // good
sayHello ()

Hello World!


or to add a unit parameter to the function

In [20]:
let sayHello () = printfn "Hello World!" // good
sayHello ()

Hello World!


## Tuples vs. Multiple Parameters

In [21]:
// a function that takes two distinct parameters
let addTwoParams x y = x + y
sgn addTwoParams

Function: x:int -> y:int -> int

In [22]:
// another function that takes a single tuple parameter but looks like it takes two ints
let addTupleParams (x,y) = x + y
sgn addTupleParams

Function: tupledArg:int * int -> int

Let’s look at the signatures (it is always a good idea to look at the signatures if you are unsure)

```fsharp
val addTwoParams : int -> int -> int        // two params
val addTupleParams : int * int -> int       // tuple->int
```

Now let’s use them:

In [23]:
addTwoParams 3 2      // ok - uses spaces to separate args

5

In [24]:
addTwoParams (1,2)    // error trying to pass a single tuple

Stopped due to error

input.fsx (1,15)-(1,18) typecheck error This expression was expected to have type
    'int'    
but here has type
    ''a * 'b'    



Cell not executed: compilation error

In [25]:
addTupleParams (5,2) 

7

See more on
- [function definitions](https://fsharpforfunandprofit.com/posts/defining-functions/)
- [function calls](https://fsharpforfunandprofit.com/posts/convenience-partial-application/)
- [function compositions](https://fsharpforfunandprofit.com/posts/convenience-functions-as-interfaces/)

## Questions

1. Write a `sayGreeting` function that takes two parameters: `greeting` and `name`.
2. Create a `hello` function from it using partial application, where the greeting set to `"Hello"`.
3. Create a `goodbye` function from it using partial application, where the greeting set to `"Goodbye"`.

In [74]:
let sayGreeting greeting name = ...
let hello = ...
let goodbye = ...

In [75]:
hello "John"  // "Hello John"

Hello John!


In [76]:
goodbye "John" // "Goodbye John"

Goodbye John!


In [ ]:
let sayGreeting greeting name = printfn "%s %s!" greeting name
let hello = sayGreeting "Hello"
let goodbye = sayGreeting "Goodbye"

*Q.* `sprintf` is a function that is similar to `printfn` except that it returns a string rather than writing to the console. Create a `sprintName` function that uses `sprintf` instead of `printfn`. What is its signature?

In [8]:
// Answer
let sprintName aName =
    sprintf "%s" aName

// val sprintName :
//    aName:string -> string

In [9]:
sprintName "test"

"test"

*Q.* Using the `let` syntax, create a function that multiplies its argument by two. What is its signature?

In [10]:
// Answer
let multiplyBy2 x = x * 2

// val multiplyBy2 :
//    x:int -> int

In [11]:
multiplyBy2 2

4

*Q.* Rewrite `let add x y = x + y` using lambdas. There are two alternatives. Compare the signatures of these alternative functions with the original.

In [ ]:
// Answer
let add_v1 x y =
    x + y
// val add_v1 : x:int -> y:int -> int

let add_v2 x =
    fun y -> x + y
// val add_v2 : x:int -> y:int -> int

let add_v3 =
    fun x y -> x + y
// val add_v3 : x:int -> y:int -> int

// alternative to v2 using an inner function
let add_v2a x =
    let innerFn y = x + y
    innerFn // return    
// val add_v2a : x:int -> y:int -> int

let add_v2b =
    fun x -> fun y -> x + y
// val add_v2b : x:int -> y:int -> int